In [1]:
import json # used for managing the JSON files from API
from urllib.request import urlopen # fetch URl of API
from pprint import pprint # just for printing values for human use

In [2]:
# Loads the papers to json
papers = json.load(urlopen("http://dirac.nist.gov/adsorption.nist.gov/isodb/api/minimalbiblio.json"))

In [25]:
authors_ids = {}
with open("./stored_authors/authors_ids.txt") as authors_file:
    authors_ids = eval(authors_file.read())
# pprint(authors_ids)

In [4]:
pprint(papers[0])

{'DOI': '10.1002/adem.200500223',
 'authors': ['75dc757852ded186f04bf13eae66c53803db5c82',
             '25e2798b6f1c8d5ca03eda52e347d33da845514f',
             '95f1bf7d79db23a55c6d8c155ecb3a6c2ec533ca',
             '7653e3bbcd863a9e7e7e142c07df15292a347874',
             '979826cedb64012e6ac7dd2787fcea904501fddc',
             '8e693fc4e5de76ff86bafd4577f7cd7a8556f74c'],
 'journal': 'Advanced Engineering Materials',
 'title': 'Improved Hydrogen Storage in the Metal-Organic Framework Cu3(BTC)2'}


In [5]:
# Just for visual clarity, not used for processing

papers_with_names = {}
for paper in papers:
    papers_with_names[ paper["DOI"] ] = []
    for person in paper["authors"]:
        papers_with_names[ paper["DOI"] ].append(authors_ids[person])
pprint(papers_with_names["10.1002/adem.200500223"])

[['P.', '', 'Krawiec'],
 ['Markus', '', 'Kramer'],
 ['M.', '', 'Sabo'],
 ['R.', '', 'Kunschke'],
 ['Heidrun', '', 'Fröde'],
 ['Stefan', '', 'Kaskel']]


In [6]:
duplicate_authors = {}
with open("./stored_authors/duplicate_authors.txt") as dup_file:
    duplicate_authors = eval(dup_file.read())

In [16]:
unknown_authors = {}
with open("./stored_authors/unknown_authors.txt") as unk_file:
    unknown_authors = eval(unk_file.read())

In [24]:
# not sure about this. Generate a coauthors dict?

coauthors = {}

# # Iterates through papers 
# for paper in papers:
    
#     # Iterates through authors
#     for person in paper["authors"]:
#         if person in unknown_authors or person in unknown_authors.values():
#             print(person)

In [23]:
# if "75dc757852ded186f04bf13eae66c53803db5c82" in unknown_authors:
#     pprint(unknown_authors)

In [10]:
# Test cell

for author_id, name in authors_ids.items():
    if "Ribeiro" in name and "Ana" in name:
        print(author_id)
print()
for paper in papers:
    if "8b80ee24d983f356b3117484ac1a6f9a2e7ce441" in paper["authors"]:
        print(paper["DOI"])

831da78505f36a7f7f27627e3cf7c7ce70f3c814
8b80ee24d983f356b3117484ac1a6f9a2e7ce441

10.1002/ceat.201300046
10.1007/s10450-014-9639-3
10.1016/j.ces.2008.07.017
10.1016/j.micromeso.2011.06.024
10.1016/j.micromeso.2011.12.043
10.1016/j.seppur.2012.02.023
10.1016/j.seppur.2012.10.009
10.1016/j.seppur.2013.03.009
10.1016/j.seppur.2015.06.012
10.1021/acs.energyfuels.5b00975


In [11]:
# Test cell

for author_id, name in authors_ids.items():
    if "Krawiec" in name:
        print(author_id)
print()
for paper in papers:
    if "75dc757852ded186f04bf13eae66c53803db5c82" in paper["authors"]:
        print(paper["DOI"])

6517995a99763fe7f2bd80ac3a5672518323910f
75dc757852ded186f04bf13eae66c53803db5c82
ORC_ID1234567890

10.1002/adem.200500223
